In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1)

In [3]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [4]:
t = Tokenizer()
t.fit_on_texts([text])

In [5]:
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d'%vocab_size)

단어 집합의 크기 : 12


In [6]:
# 단어와 단어에 부여된 정수 인덱스 출력
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [7]:
# 훈력데이터 생성
sequences = list()
for line in text.split('\n'):   # \n기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('학습에 사용할 샘플의 수 : %d'%len(sequences))

학습에 사용할 샘플의 수 : 11


In [8]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


- 상기 리스트는 label로 사용될 단어 분리되지 않은 상태
- [2,3]은 [경마장에, 있는]이며 우측의 데이터를 label로 사용
- 모든 상품에 대해 길이 일치 : 즉 가장 긴 샘플의 길이를 찾자

In [9]:
max_len = max(len(l) for l in sequences)
print("최대 길이:{}".format(max_len))

최대 길이:6


- 모든 샘플에 대해서 0으로 패딩하여 길이 맞춰줌

In [10]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
# pad_sequences() 모든 샘플에 대해서 "0"으로 패딩
# maxlen : 길이 설정
# padding='pre' : 앞쪽으로 패딩

In [11]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


- Label 분리

In [12]:
sequences = np.array(sequences)
X = sequences[:,:-1]
Y = sequences[:,-1]
print(X)
print(Y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


- Label에 대해서 원-핫 인코딩

In [13]:
y = to_categorical(Y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 모델설계하기

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))  
# vocab_size : 토큰화된 단어들의 개수
# 10 : 원핫벡터 임베딩 차원
# input_length : 인풋의 길이
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
 - 0s - loss: 2.4581 - acc: 0.1818
Epoch 2/200
 - 0s - loss: 2.4445 - acc: 0.2727
Epoch 3/200
 - 0s - loss: 2.4308 - acc: 0.2727
Epoch 4/200
 - 0s - loss: 2.4168 - acc: 0.2727
Epoch 5/200
 - 0s - loss: 2.4025 - acc: 0.3636
Epoch 6/200
 - 0s - loss: 2.3877 - acc: 0.4545
Epoch 7/200
 - 0s - loss: 2.3724 - acc: 0.4545
Epoch 8/200
 - 0s - loss: 2.3566 - acc: 0.4545
Epoch 9/200
 - 0s - loss: 2.3402 - acc: 0.4545
Epoch 10/200
 - 0s - loss: 2.3232 - acc: 0.4545
Epoch 11/200
 - 0s - loss: 2.3056 - acc: 0.4545
Epoch 12/200
 - 0s - loss: 2.2873 - acc: 0.4545
Epoch 13/200
 - 0s - loss: 2.2684 - acc: 0.4545
Epoch 14/200
 - 0s - loss: 2.2489 - acc: 0.4545
Epoch 15/200
 - 0s - loss: 2.2289 - acc: 0.4545
Epoch 16/200
 - 0s - loss: 2.2085 - acc: 0.4545
Epoch 17/200
 - 0s - loss: 2.1877 - acc: 0.3636
Epoch 18/200
 - 0s - loss: 2.1667 - acc: 0.3636
Epoch 19/200
 - 0s - loss

Epoch 160/200
 - 0s - loss: 0.2653 - acc: 1.0000
Epoch 161/200
 - 0s - loss: 0.2607 - acc: 1.0000
Epoch 162/200
 - 0s - loss: 0.2561 - acc: 1.0000
Epoch 163/200
 - 0s - loss: 0.2515 - acc: 1.0000
Epoch 164/200
 - 0s - loss: 0.2471 - acc: 1.0000
Epoch 165/200
 - 0s - loss: 0.2427 - acc: 1.0000
Epoch 166/200
 - 0s - loss: 0.2383 - acc: 1.0000
Epoch 167/200
 - 0s - loss: 0.2341 - acc: 1.0000
Epoch 168/200
 - 0s - loss: 0.2299 - acc: 1.0000
Epoch 169/200
 - 0s - loss: 0.2257 - acc: 1.0000
Epoch 170/200
 - 0s - loss: 0.2216 - acc: 1.0000
Epoch 171/200
 - 0s - loss: 0.2176 - acc: 1.0000
Epoch 172/200
 - 0s - loss: 0.2136 - acc: 1.0000
Epoch 173/200
 - 0s - loss: 0.2098 - acc: 1.0000
Epoch 174/200
 - 0s - loss: 0.2059 - acc: 1.0000
Epoch 175/200
 - 0s - loss: 0.2022 - acc: 1.0000
Epoch 176/200
 - 0s - loss: 0.1984 - acc: 1.0000
Epoch 177/200
 - 0s - loss: 0.1948 - acc: 1.0000
Epoch 178/200
 - 0s - loss: 0.1912 - acc: 1.0000
Epoch 179/200
 - 0s - loss: 0.1877 - acc: 1.0000
Epoch 180/200
 - 0s 

## 예측 (문장을 생성하여 출력)

In [16]:
def sentence_generation(model, t, current_word, n):  # 모델, 토크나이저, 현재 단어, 반복횟수
    init_word = current_word  # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    for _ in range(n):   # n번 반복
        encoded = t.texts_to_sequences([current_word])[0]   # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        
    #입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장
        for word, index in t.word_index.items():
            if index == result:   # 예측한 단어와 인덱스와 동일한 단어가 있다면
                break   # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word  # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word    # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [18]:
print(sentence_generation(model, t, '경마장에', 4))   # '경마장에'라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

경마장에 있는 말이 뛰고 있다


In [19]:
print(sentence_generation(model, t, '그의', 2))     # 2번 예측

그의 말이 법이다


In [21]:
print(sentence_generation(model, t, '가는', 5))      # 5번 예측

가는 말이 고와야 오는 말이 곱다
